In [3]:
#Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV

In [11]:
#Levantamos el df_final2 y df_final_alternativo
df_final2 = pd.read_csv('df_final2.csv')
df_final_alternativo = pd.read_csv('df_final_alternativo.csv')

In [12]:
#Generamos el X y el target
#X = df_final2.drop(['target'], axis = 1)
#y = df_final2['target']

#Generamos el X y el target
Xa = df_final_alternativo.drop(['ClasificacionNum2', 'ID'], axis = 1)
ya = df_final_alternativo['ClasificacionNum2']

#Divimos en train y en test
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa,ya, stratify=ya, test_size= 0.25, random_state=10)

#Estandarizamos con MinMax
scaler = MinMaxScaler()

Xa_train_std = scaler.fit_transform(Xa_train)
Xa_test_std = scaler.fit_transform(Xa_test)

In [6]:
#Dividimos en train y en test
#X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size= 0.25)

In [7]:
#Estandarizamos las variables usando 3 formas diferentes

#scaler = StandardScaler()
#scaler2 = RobustScaler()
#scaler3 = MinMaxScaler()

#Standard
#X_train_std = scaler.fit_transform(X_train)
#X_test_std = scaler.fit_transform(X_test)

#Robust
#X_train_std_robust = scaler2.fit_transform(X_train)
#X_test_std_robust = scaler2.fit_transform(X_test)

#MinMax
#X_train_std_mM = scaler3.fit_transform(X_train)
#X_test_std_mM = scaler3.fit_transform(X_test)

In [14]:
#Con GridSearch buscamos los mejores hiperparámetros

leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Hacemos un diccionario

hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Instanciamos el modelo de KNN
knn = KNeighborsClassifier()

#Usamos GridSearch
clf = GridSearchCV(knn, hyperparameters, cv=10)

#Ajustamos el modelo
best_model = clf.fit(Xa_train_std,ya_train)

#Pinteamos los mejores hiperparámetros
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 1
Best p: 1
Best n_neighbors: 14


In [15]:
#Instanciamos con los mejores hiperparámetros y hacemos cross validation
knn = KNeighborsClassifier (leaf_size= best_model.best_estimator_.get_params()['leaf_size'], p= best_model.best_estimator_.get_params()['p'] , n_neighbors= best_model.best_estimator_.get_params()['n_neighbors'])
stratifiedkf=StratifiedKFold(n_splits=20)
score=cross_val_score(knn,Xa_train_std,ya_train,cv=stratifiedkf)

print("Los scores obtenidos son {}".format(score))
print("El promedio es :{}".format(score.mean()))
print("La desviación es :{}".format(np.std(score)))

Los scores obtenidos son [0.73333333 0.7        0.66666667 0.73333333 0.63333333 0.86666667
 0.73333333 0.66666667 0.65517241 0.62068966 0.72413793 0.72413793
 0.75862069 0.75862069 0.72413793 0.62068966 0.75862069 0.72413793
 0.72413793 0.72413793]
El promedio es :0.7125287356321838
La desviación es :0.056139630200122206


In [ ]:
knn.fit(X_train_std_mM, y_train)
y_pred = knn.predict(X_test_std_mM)
y_pred